<a href="https://colab.research.google.com/github/kiyong21c/chobocoding/blob/master/20220610_cat-in-dat_improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 성능 개선
 - 피처 엔지니어링 : 피처 맞춤 인코딩, 피처 스케일링
 - 하이퍼 파라미터 최적화 : 그리드 서치

 - 1. 인코딩을 피처 특성에 맞게 적용해야 함
 
  > 이진 피처와 순서형 피처 ord_1, ord_2는 수작업 인코딩

  > 순서형 피처 ord_3, ord_4, ord_5는 ordinal 인코딩

  > 명목형 피처와 날짜 피처는 원-핫 인코딩

 - 2. 피처 스케일링 적용(피처 간 값의 범위를 일치)
 
  > 순서형 피처에만 피처 스케일링 적용

  > 이진 피처, 명목형 피처, 날짜 피처는 인코딩 후 이미 최솟값:0, 최댓값:1로 범위가 일치하기 때문에 스케일링 필요 없음

 - 3. 하이퍼파리미터 최적화

In [4]:
import pandas as pd

data_path = '/content/drive/MyDrive/Colab Notebooks/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 피처 엔지니어링 1 : 피처 맞춤 인코딩
 - 모든 피처를 일괄적으로 원-핫 인코딩 하는것 보다 피처 특성에 맞게 인코딩하면 성능 개선

#### 데이터 합치기

In [5]:
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃값 제거

#### 이진 피처 인코딩
 - bin_0, bin_1, bin_2 피처는 이미 0, 1로 구성
 - bin_3, bin_4 피처는 T, F, Y, N 으로 구성 : 각각 1, 0으로 변경

In [6]:
# Series 객체에 map() 함수를 호출하면, Series의 모든 원소에 적용해 결과 반환
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

#### 순서형 피처 인코딩

In [13]:
ord_features = ['ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']

for i, feature in enumerate(ord_features):
    print(f'{feature} 고윳값 : {all_data[feature].unique()}')

ord_0 고윳값 : [2 1 3]
ord_1 고윳값 : ['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']
ord_2 고윳값 : ['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']
ord_3 고윳값 : ['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']
ord_4 고윳값 : ['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']
ord_5 고윳값 : ['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' 'To' 'uy' 'ZS' 'vy' 'ZR' 'AP' 'GJ'
 'Wv' 'ri' 'qw' 'Xh' 'FI' 'nh' 'KR' 'dB' 'BE' 'Bb' 'mc' 'MC' 'tM' 'N

 - ord_0 피처는 이미 숫자 : 인코딩 필요X
 - ord_1, ord_2 : 숫자를 정해서 인코딩
 - ord_3 ~ ord_5 : 알파벳 순서로 인코딩

In [ ]:
ord1dict = {'Novice':0, 'Contributor':1, 'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, ''}